# Ejercicio 28

El porcentaje de contenidos que están publicados cuya última edición no tiene
comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
df_contents = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.parquet', header=True, inferSchema=True)


In [ ]:
rdd_contents = df_contents.rdd

In [ ]:
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

## Opcion 1

In [ ]:
null_comments_by_revisors = rdd_contents.filter(lambda x: x.revisor_comment == None).map(lambda x: (x.revisor_username,1) if x.revisor_username != None else (x.revisor_ip,1))\
                                      .reduceByKey(lambda x,y: x+y)

In [ ]:
total_coments_by_revisors = rdd_contents.map(lambda x: (x.revisor_username,1) if x.revisor_username != None else (x.revisor_ip,1))\
                                      .reduceByKey(lambda x,y: x+y)

In [ ]:
total_coments_by_revisors.take(5)

[('2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', 13),
 ('Lojwe', 24169),
 ('Jialxv', 2905),
 ('PatriHorrillo', 831),
 ('SeroBOT', 35412)]

In [ ]:
null_and_total_comments_by_revisors = null_comments_by_revisors.fullOuterJoin(total_coments_by_revisors).cache()

In [ ]:
null_and_total_comments_by_revisors.take(5)

[('Arctos71', (38, 47)),
 ('TueorUe', (35, 39)),
 ('2806:1000:8101:7AFD:4962:AE:51CB:EFFD', (2, 2)),
 ('Telephone Directory', (217, 229)),
 ('Polinizador', (12413, 18924))]

In [ ]:
null_and_total_comments_by_revisors.filter(lambda x: x[1][0] == None).take(10)

[('2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', (None, 13)),
 ('InternetArchiveBot', (None, 61756)),
 ('190.164.54.234', (None, 2)),
 ('Leptobot', (None, 9524)),
 ('Jose Oteros', (None, 3)),
 ('Odalcet', (None, 22)),
 ('David Perez', (None, 396)),
 ('200.87.95.239', (None, 3)),
 ('2806:108E:15:1718:DD00:E422:E54:99B7', (None, 1)),
 ('81.0.38.70', (None, 1))]

In [ ]:
null_and_total_comments_by_revisors.filter(lambda x: x[1][1] == None).take(10)
# tiene sentido que no haya porque no puede no tener ediciones pero 
# tener una edición sin comentario. Lo chequeo por las dudas.

[]

In [ ]:
null_and_total_comments_by_revisors = null_and_total_comments_by_revisors.map(lambda x: (x[0],(0,x[1][1])) if x[1][0] == None else (x[0],(x[1][0],x[1][1]))).cache()

In [ ]:
null_and_total_comments_by_revisors.filter(lambda x: x[1][0] == 0).take(5)

[('2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', (0, 13)),
 ('InternetArchiveBot', (0, 61756)),
 ('190.164.54.234', (0, 2)),
 ('Leptobot', (0, 9524)),
 ('Jose Oteros', (0, 3))]

In [ ]:
one_edition_percentage = null_and_total_comments_by_revisors.filter(lambda x: x[1][1] == 10).map(lambda x: x[1][0]).reduce(lambda x,y: x+y) / \
null_and_total_comments_by_revisors.filter(lambda x: x[1][1] == 10).map(lambda x: x[1][1]).reduce(lambda x,y: x+y)

In [ ]:
one_edition_percentage = one_edition_percentage*100
one_edition_percentage

29.812925170068027

In [ ]:
above_ten_edition_percentage = null_and_total_comments_by_revisors.filter(lambda x: x[1][1] > 10).map(lambda x: x[1][0]).reduce(lambda x,y: x+y) / \
null_and_total_comments_by_revisors.filter(lambda x: x[1][1] > 10).map(lambda x: x[1][1]).reduce(lambda x,y: x+y)

In [ ]:
above_ten_edition_percentage = above_ten_edition_percentage*100
above_ten_edition_percentage

10.140649236405128

In [ ]:
above_hundred_edition_percentage = null_and_total_comments_by_revisors.filter(lambda x: x[1][1] > 100).map(lambda x: x[1][0]).reduce(lambda x,y: x+y) / \
null_and_total_comments_by_revisors.filter(lambda x: x[1][1] > 100).map(lambda x: x[1][1]).reduce(lambda x,y: x+y)

In [ ]:
above_hundred_edition_percentage = above_hundred_edition_percentage*100
above_hundred_edition_percentage

9.365573554956518

# Respuesta

In [ ]:
print("El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1 edición es",one_edition_percentage,", >10 es",above_ten_edition_percentage,"y >100 es",above_hundred_edition_percentage)

El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1 edición es 29.812925170068027 , >10 es 10.140649236405128 y >100 es 9.365573554956518
